In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm 

import xicorpy                  # Pour utiliser les estimateurs des indices de Cramèr-von-Mises
from sklearn import tree        # Pour utiliser les arbres de classification

In [ ]:
# Pour éviter quelques erreurs d'affichages suite à la conversion numpy/pandas en utilisant le package xicorpy
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# **1) Classification sans l'Analyse de Sensibilité**

Nous allons travailler avec le modèle théorique :
$$ Y =  X^{(1)} \times \dots \times X^{(d)} $$

Nous cherchons à classifier selon le signe de $Y$. Il y a donc trois classes : $\left\{-1, \ 0, \ 1\right\}$

In [ ]:
def MakeSimulator( dim_d ):
    return lambda inputs : np.prod( inputs[ : dim_d, : ], axis = 0 ) 

dim_d     = 5
simulator = MakeSimulator( dim_d )

Nous allons noyer les $d$ variables dans $D \ge d$ variables indépendantes. Chacun des $X^{(i)}$ sera uniforme sur $[0,1]$. Regardons d'abord ce qu'il se passe pour $d=D$.

In [ ]:
dim_D = 5

n_train = int( 7.5e4 )
n_test  = int( 2.5e4 )

inputs_train = np.random.uniform( low = -1, high = 1, size = ( dim_D, n_train ) )
inputs_test  = np.random.uniform( low = -1, high = 1, size = ( dim_D, n_train ) )
output_train = simulator( inputs_train )
output_test  = simulator( inputs_test )


1) Utiliser scikit-learn pour importer un algorithme générant un arbre de classification. L'utiliser pour entrainer un arbre sur les données d'entrainement, puis calculer le score du modèle sur les données de test.
(voir https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

2. Faire maintenant varier le nombre de variable totale de $D=5$ à $D=20$. Observer le score du classifieur sur un scatter plot, à mesure que $D$ augmente.

In [ ]:
dim_D_max = 20
score_list_no_gsa = []



In [ ]:
plt.figure( figsize = ( 10, 5 ) )
plt.scatter( np.arange( dim_d, dim_D_max ), score_list_no_gsa, color = 'blue', label = 'Score du Classifieur' )
plt.title( 'Score du classifieur quand la dimension augmente', fontsize = 16 )
plt.xlabel( u'Nombre de variable au total $D$', fontsize = 14) 
plt.ylabel( u'Score $R^2$', fontsize = 14 )
plt.legend( fontsize = 10 )
plt.show()

3. Le score pour des "grandes" valeurs de $D$ est-il prévisible, ou du moins explicable ?

4. Produire un scatter plot représentant le temps d'entrainement à mesure que le nombre de variable augmente. On prendra de nouveau $D$ entre 5 et 20.

In [ ]:
dim_D_max = 20
time_list_no_gsa = []



In [ ]:
plt.figure( figsize = ( 10, 5 ) )
plt.scatter( np.arange( dim_d, dim_D_max ), time_list_no_gsa, color = 'blue', label = "Temps d'entrainement" )
plt.title( "Temps d'entrainement du classifieur quand la dimension augmente", fontsize = 16 )
plt.xlabel( u'Nombre de variable au total $D$', fontsize = 14) 
plt.ylabel( u'Temps en seconde', fontsize = 14 )
plt.legend( fontsize = 10 )
plt.show()

# **2) Intervention de l'Analyse de Sensibilité**

On va utiliser l'estimation des indices de Cramèr-von-Mises pour réduire le nombre de variable.

5) Revenir au cas $d = D = 5$ et évaluer les indices de Cramèr-von-Mises pour chaque variable.
(voir https://swarnakumar.github.io/xicorpy/xi/ et pour plus de détail sur l'api : https://swarnakumar.github.io/xicorpy/xicor_api/)

6. Quelle information semble ressortir de cette analyse à propos des effets d'ordre supérieur ?

On rappelle que d'après le TCL pour l'estimateur $\widehat{S}_{\mathrm{CvM}}^{(i)}$ des indices de CvM, sous l'hypothèse que $X^{(i)}$ est indépendante de $$Y$, on a :
$$ \sqrt{ \frac{5n}{2} }\widehat{S}_{\mathrm{CvM}}^{(i)} \rightarrow \mathcal{N}(0,1) $$
Ainsi si la réalisation de $\widehat{S}_{\mathrm{CvM}}^{(i)}$ dépasse $1.96 \times \sqrt{ \frac{2}{5n} }$, alors on peut considérer la variable $X^{(i)}$ pertinente avec un risque de $5\%$ (test unilatéral).

7. Reprendre la classifcation et le calcul de score effectué à la question 2, en sélectionnant uniquement les variables, dont l'estimateur des indices de CvM dépassent le seuil donné ci-dessus par le TCL. En faire aussi un scatter plot et le comparer à celui établi à la question 2.

In [ ]:
dim_D_max = 20
score_list_gsa = []



In [ ]:
plt.figure( figsize = ( 10, 5 ) )
plt.scatter( np.arange( dim_d, dim_D_max ), score_list_no_gsa, color = 'blue', label = 'Sans Analyse de Sensibilité' )
plt.scatter( np.arange( dim_d, dim_D_max ), score_list_gsa, color = 'red', label = 'Avec Analyse de Sensibilité' )
plt.title( 'Score du classifieur quand la dimension augmente', fontsize = 16 )
plt.xlabel( u'Nombre de variable au total $D$', fontsize = 14) 
plt.ylabel( u'Score $R^2$', fontsize = 14 )
plt.legend( fontsize = 10 )
plt.show()

8. Reprendre la classifcation et l'évaluation du temps d'entrainement effectué à la question 4, en sélectionnant uniquement les variables, dont l'estimateur des indices de CvM dépassent le seuil donné ci-dessus par le TCL. En faire aussi un scatter plot et le comparer à celui établi à la question 4.

In [ ]:
dim_D_max = 20
time_list_gsa = []



In [ ]:
plt.figure( figsize = ( 10, 5 ) )
plt.scatter( np.arange( dim_d, dim_D_max ), time_list_no_gsa, color = 'blue', label = 'Sans Analyse de Sensibilité' )
plt.scatter( np.arange( dim_d, dim_D_max ), time_list_gsa, color = 'red', label = 'Avec Analyse de Sensibilité' )
plt.title( "Temps d'entrainement du classifieur quand la dimension augmente", fontsize = 16 )
plt.xlabel( u'Nombre de variable au total $D$', fontsize = 14) 
plt.ylabel( u"Temps d'entrainement en seconde", fontsize = 14 )
plt.legend( fontsize = 10 )
plt.show()